In [1]:
from utils.data import from_dir, random_resample

data_path = 'data/chest_xray/train'

In [2]:
resize = (96, 96)

In [3]:
training_selector = lambda s: s.startswith('chest_xray/train/') and s.endswith('.jpeg')
(x, y) = from_dir(data_path, resize=resize, resample=random_resample)

In [4]:
NEG_IDX = 0
POS_IDX = 1
TOTAL   = len(y)
N       = len(list(filter(lambda j: j == 0, y)))
P       = len(list(filter(lambda j: j == 1, y)))
print(f'#Positives = {P} ({P/TOTAL*100:.2f}%)')
print(f'#Negatives = {N} ({N/TOTAL*100:.2f}%)')

#Positives = 3883 (74.22%)
#Negatives = 1349 (25.78%)


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(x, y, test_size=0.2, random_state=42)
total_train = len(Y_train)
N_train     = len(list(filter(lambda j: j == 0, Y_train)))
P_train     = len(list(filter(lambda j: j == 1, Y_train)))
print(f'#Positives (train) = {P_train} ({P_train/total_train*100:.2f}%)')
print(f'#Negatives (train) = {N_train} ({N_train/total_train*100:.2f}%)')
total_val   = len(Y_val)
N_val       = len(list(filter(lambda j: j == 0, Y_val)))
P_val       = len(list(filter(lambda j: j == 1, Y_val)))
print(f'#Positives (val)   = {P_val} ({P_val/total_val*100:.2f}%)')
print(f'#Negatives (val)   = {N_val} ({N_val/total_val*100:.2f}%)')

#Positives (train) = 3095 (73.95%)
#Negatives (train) = 1090 (26.05%)
#Positives (val)   = 788 (75.26%)
#Negatives (val)   = 259 (24.74%)


In [6]:
from keras import callbacks as cb, losses, metrics as metr, optimizers
from models import loop, create_model

callbacks = [cb.EarlyStopping(patience=5, verbose=1)]
model = create_model('resnet-18', resize)
metrics = [metr.BinaryCrossentropy(name='loss'), metr.BinaryCrossentropy('val_loss'), metr.Recall(name='recall'), metr.Recall(name='val_recall')]

history = loop.training_loop(
    X_train, Y_train, X_val, Y_val,
    model,
    4,
    64,
    metrics,
    optimizers.Adam(),
    callbacks,
    losses.BinaryCrossentropy(name='loss'))
history

2025-02-23 21:05:25.503009: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-23 21:05:25.503319: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-23 21:05:25.504859: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-23 21:05:25.508235: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740341125.513831  116502 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740341125.51

Epoch 1/4
 1/64 ━━━━━━━━━━━━━━━━━━━━ 2:09 2s/stepTraining loss (for one batch) at step 0: 0.7414
Seen so far: 64 samples
201/64 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ -21s 159ms/stepTraining loss (for one batch) at step 200: 0.1202
Seen so far: 12864 samples
401/64 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ -52s 157ms/stepTraining loss (for one batch) at step 400: 0.1114
Seen so far: 25664 samples
532/64 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ -73s 157ms/step

KeyboardInterrupt: 

In [8]:
history.history

{'loss': [0.31221696734428406,
  0.1416202038526535,
  0.10713862627744675,
  0.10334759205579758],
 'val_loss': [2.948640823364258,
  3.5781400203704834,
  2.796703338623047,
  3.078964948654175]}